In [1]:
import numpy as np
from pathlib import Path
import yaml
import cv2 as cv
from sklearn.model_selection import train_test_split

In [2]:
def prev_im(image, string=""):
    """
    Display an image in a window with OpenCV.

    Parameters:
    ---
    image : numpy.ndarray
        The image to be displayed, expected to be in the format readable by OpenCV (usually uint8, with dimensions [height, width, channels]).
    string : str, optional
        The title of the window in which the image will be displayed. Default is an empty string.

    Returns:
    ---
    None
    """
    cv.imshow(string, image)  # Display the image in a window with the title provided in 'string'.
    cv.waitKey(0)            # Wait indefinitely until a key is pressed.
    cv.destroyAllWindows()   # Close all OpenCV windows.

In [3]:
# Image Processing - Grayscale Conversion
def gen_gray_img(map_data: np.ndarray, debug=False):
    """
    Generate a gray-scale image from wafer data.

    Parameters:
    ---
    map_data : numpy.ndarray
        Wafer data map with pixel values of (0, 1, 2); 0=non-wafer, 1=good wafer, 2=bad wafer.
    debug : bool, optional
        If True, performs checks and logs issues instead of raising exceptions.

    Returns:
    ---
    numpy.ndarray
        Gray-scale image with pixel values (0, 128, 255).
    """
    expected_values = {0, 1, 2}
    unique_values = np.unique(map_data)
    
    if not expected_values.issuperset(unique_values):
        if debug:
            print(f"Unexpected values found in map_data: {unique_values}")
            return None
        else:
            raise ValueError(f"Map values do not match expected values (0, 1, 2)")

    gray = np.copy(map_data)
    gray[gray == 1] += 127
    gray[gray == 2] += 253  # direct assignment to max value for clarity

    return gray.astype(np.uint8)

In [4]:
# Image Processing - Colour Encoding
def gray_to_color(gray_map:np.ndarray,
                  bad_clr:tuple=(255,255,0),
                  good_clr:tuple=(25,102,255)):
    """
    Usage
    ---
    Convert gray-scale image to color image using `good_clr` and `bad_clr` BGR colors

    Parameters
    ---
    gray_map : ``numpy.ndarray``
    bad_clr : ``tuple`` optional,
        BGR color values to use for 'bad' pixels, `default=(255,255,0)` "Pumpkin"
    good_clr : ``tuple`` optional,
        BGR color values to use for 'good' pixels, `default=(25,102,255)` "Aqua"

    Returns
    ---
    BGR color image of wafer map, using `good_clr` and `bad_clr` pixel values

    """
    assert all([n in np.unique(gray_map) for n in [0,128,255]]), f"Gray scale values do not match expected values (0, 128, 255)"

    color_map = cv.cvtColor(np.copy(gray_map),cv.COLOR_GRAY2BGR) # B, G, R
    
    for d in range(color_map.shape[-1]):
            color_map[:,:,d][color_map[:,:,d] == 255] = bad_clr[d]
            color_map[:,:,d][color_map[:,:,d] == 128] = good_clr[d]
    
    return color_map

In [5]:
# Data Loading
import pandas as pd
data = np.load("/kaggle/input/mixedtype-wafer-defect-datasets/Wafer_Map_Datasets.npz")

In [6]:
# Seperate wafer map data and labels
imgs = data['arr_0'] # shape (38015, 52, 52)
lbls = data['arr_1'] # shape (38015, 8)

# Get unique labels
unique_lbls = np.unique(lbls,axis=0)

In [7]:
# Convert each label row to a tuple for it to be hashable
label_tuples = [tuple(label) for label in lbls]

# Use a dictionary to count occurrences of each unique label
label_count = {}
for label in label_tuples:
    if label in label_count:
        label_count[label] += 1
    else:
        label_count[label] = 1

# Display the counts
for label, count in label_count.items():
    print(f"Label {label}: {count} images")

Label (1, 0, 1, 0, 0, 0, 1, 0): 2000 images
Label (1, 0, 1, 0, 0, 0, 0, 0): 1000 images
Label (1, 0, 0, 1, 0, 0, 1, 0): 1000 images
Label (1, 0, 0, 1, 0, 0, 0, 0): 1000 images
Label (1, 0, 1, 0, 1, 0, 1, 0): 1000 images
Label (1, 0, 1, 0, 1, 0, 0, 0): 1000 images
Label (1, 0, 0, 1, 1, 0, 1, 0): 1000 images
Label (1, 0, 0, 1, 1, 0, 0, 0): 1000 images
Label (1, 0, 0, 0, 1, 0, 1, 0): 1000 images
Label (1, 0, 0, 0, 1, 0, 0, 0): 1000 images
Label (1, 0, 0, 0, 0, 0, 1, 0): 1000 images
Label (1, 0, 0, 0, 0, 0, 0, 0): 1000 images
Label (0, 1, 1, 0, 0, 0, 1, 0): 1000 images
Label (0, 1, 1, 0, 0, 0, 0, 0): 1000 images
Label (0, 1, 0, 1, 0, 0, 1, 0): 1000 images
Label (0, 1, 0, 1, 0, 0, 0, 0): 1000 images
Label (0, 1, 1, 0, 1, 0, 1, 0): 1000 images
Label (0, 1, 1, 0, 1, 0, 0, 0): 1000 images
Label (0, 1, 0, 1, 1, 0, 1, 0): 1000 images
Label (0, 1, 0, 1, 1, 0, 0, 0): 1000 images
Label (0, 1, 0, 0, 1, 0, 1, 0): 1000 images
Label (0, 1, 0, 0, 1, 0, 0, 0): 1000 images
Label (0, 1, 0, 0, 0, 0, 1, 0): 

In [8]:
# Load encoding file and create string labels
# YAML file provides a structured way to map arrays of numeric labels to human-readable strings, which describe various types of wafer defects.
encodes_path = "/kaggle/input/encodings-yaml/encodings.yaml"
with open(encodes_path, 'r') as enc:
    encd = yaml.safe_load(enc)

str_lbls = [str(l) for l in lbls]

In [9]:
# Show the number of unique pixel values in the uncleaned data
unique_pixels = set()  # To store all unique pixel values across all images
bad_wafers = []        # To store images that are considered bad based on pixel values

# Analyze each wafer image
for img in imgs:
    vals = np.unique(img).tolist()  # Get unique pixel values in the current image
    unique_pixels.update(vals)      # Update the set of unique pixel values

    # Check for unexpected pixel values
    if len(vals) > 3:
        bad_wafers.append(img)      # Append to bad wafers if unexpected values are found

# Display the results
print(f"Unique pixel values before cleaning: {unique_pixels}")
print(f"Number of bad wafers: {len(bad_wafers)}")

Unique pixel values before cleaning: {0, 1, 2, 3}
Number of bad wafers: 105


In [10]:
# Fix imgs with more than 3 pixel values
# should ONLY have 0,1,2 for values
# see https://github.com/Junliangwangdhu/WaferMap/issues/2
for im in imgs:
    val = np.unique(im)
    if len(val) > 3:
        im[im == 3] = 2

In [11]:
# Train-Test split
# y is generated by mapping the labels (lbls) to their corresponding encoded keys using a dictionary (encd).
X, y = imgs, [list(encd.keys())[list(encd.values()).index(k.tolist())] for k in lbls] 

# Split data into test and temp sets
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

# Further split the temp set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.2, random_state=10)

In [12]:
# Review counts to analyze the distribution of labels across your training, validation, and test sets after the data split.
train_lbl_counts = {k:v for k,v in zip(*np.unique(y_train,return_counts=True))}
val_lbl_counts = {k:v for k,v in zip(*np.unique(y_val,return_counts=True))}
test_lbl_counts = {k:v for k,v in zip(*np.unique(y_test,return_counts=True))}

In [13]:
import os

# Define directory paths
train_dir = '/kaggle/working/data/train'
val_dir = '/kaggle/working/data/val'
test_dir = '/kaggle/working/data/test'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [14]:
# Train
# Directory setup for each class, image processing, image resizing, saving the processed image
for ct, t in enumerate(X_train):
    cls_dir = f"{train_dir}/{y_train[ct]}"
    if not Path(cls_dir).exists():
        _ = Path.mkdir(Path(cls_dir))
    else:
        pass
    filename = f"{cls_dir}/{ct}.png"
    color = gray_to_color(gen_gray_img(t))
    stride_szd = cv.resize(np.copy(color),(64,64),interpolation=cv.INTER_CUBIC)
    _ = cv.imwrite(filename,stride_szd)

In [15]:
# Validate
# Directory setup for each class, image processing, image resizing, saving the processed image
for cval,val in enumerate(X_val):
    cls_dir = f"{val_dir}/{y_val[cval]}"
    if not Path(cls_dir).exists():
        _ = Path.mkdir(Path(cls_dir))
    else:
        pass
    filename = f"{cls_dir}/{cval}.png"
    color = gray_to_color(gen_gray_img(val))
    stride_szd = cv.resize(np.copy(color),(64,64),interpolation=cv.INTER_CUBIC)
    _ = cv.imwrite(filename,stride_szd)

In [16]:
# Test
# Directory setup for each class, image processing, image resizing, saving the processed image
for ctest,test in enumerate(X_test):
    cls_dir = f"{test_dir}/{y_test[ctest]}"
    if not Path(cls_dir).exists():
        _ = Path.mkdir(Path(cls_dir))
    else:
        pass
    filename = f"{cls_dir}/{ctest}.png"
    color = gray_to_color(gen_gray_img(test))
    stride_szd = cv.resize(np.copy(color),(64,64),interpolation=cv.INTER_CUBIC)
    _ = cv.imwrite(filename,stride_szd)

In [17]:
# Generate color images for ALL wafer maps and save images to labeled directory
for k, v in encd.items():
    # Define the directory path
    directory_path = f'./wafers/{k}'
    
    # Check if the parent directory exists, if not create it
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    
    # Get indices of matching groups
    values, *_ = np.where(np.array(str_lbls) == str(v).replace(',', ''))

    for idx in values:
        # Create grayscale image
        gray = gen_gray_img(imgs[idx])

        # Convert to color
        color = gray_to_color(gray)

        # Resize for YOLO model, (64 x 64); multiple of model stride 32
        stride_szd = cv.resize(np.copy(color), (64, 64), interpolation=cv.INTER_CUBIC)
        _ = cv.imwrite(f'./wafers/{k}/' + str(idx) + '.png', stride_szd)

In [18]:
!pip install ultralytics
from ultralytics import YOLO
from pathlib import Path
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 13.2 MB/s eta 0:00:00a 0:00:01


In [19]:
# Load a model
model = YOLO('yolov8x-cls.yaml')  
data_dir = Path("/kaggle/working/data")

model.to('cuda') if model.device.type == 'cpu' else None #moves the model to a CUDA-enabled GPU

YOLOv8x-cls summary: 183 layers, 57422840 parameters, 57422840 gradients, 155.3 GFLOPs


YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act):

In [20]:
print(os.getcwd())

/kaggle/working


In [ ]:
# Train the model
model.train(data=data_dir,
            epochs=150,
            batch=16,
            imgsz=64,
            device=0,
            workers=12,
            project='wafer_defects',
            name='EXP00006',
            seed=10,
            deterministic=True,
            val=True,
            # save_json=True,
            # save_conf=True,
            dropout=0.15,   # default 0.0
            mosaic=0.96,    # default 1.0
            # flipud=0.0,     # default 0.0
            # fliplr=0.5,     # default 0.5
            # scale=0.5,      # default 0.5
            # translate=0.1,  # default 0.1
            degrees=50,     # default 0.0
            # mixup=0.0,      # default 0.0
            # copy_paste=0.0  # default 0.0
            patience=50,
            pretrained=False,
            optimizer='SGD',
            close_mosaic=0,
            plots=True
            )

Ultralytics YOLOv8.2.28 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.yaml, data=/kaggle/working/data, epochs=150, time=None, patience=50, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=0, workers=12, project=wafer_defects, name=EXP00006, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=10, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.15, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

2024-06-04 07:32:04,974	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-04 07:32:05,789	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=1000 with nc=38

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   7375360  ultralyt

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...


      Epoch    GPU_mem       loss  Instances       Size


     68/150      1.32G     0.5218          8         64: 100%|██████████| 1331/1331 [01:13<00:00, 18.05it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 167/167 [00:03<00:00, 42.60it/s]

                   all      0.981          1



      Epoch    GPU_mem       loss  Instances       Size


     69/150      1.34G     0.5335         16         64:  21%|██        | 273/1331 [00:15<01:02, 16.94it/s]

In [27]:
# Deploy model that has been previously trained
weights = "/kaggle/working/wafer_defects/EXP00006/weights/best.pt"
model = YOLO(weights)  

# Setting data directory
data_dir = Path("/kaggle/working/data")

# Evaluate the model on the test data split
model.val(data=data_dir,split='test',save_json=True,plots=True)

# Metric extraction
cls_count = model.metrics.confusion_matrix.nc
cls_names = model.names
matrix = model.metrics.confusion_matrix.matrix 

tp, fp = model.metrics.confusion_matrix.tp_fp() # true positive and false positive (total of true class incorrect)
fn = np.array([(matrix[:,c:c+1].sum() - matrix[c,c]) for c in range(cls_count)]) # false negative, (total of predicted class incorrect)

# Per-class metric calculation
class_metrics = dict()
all_p, all_r, all_f1, all_acc = [], [], [], []
for c in range(cls_count):
    recall = tp[c] / (tp[c] + fn[c])
    precision = tp[c] / (tp[c] + fp[c])
    accuracy = (tp[c] + 0) / (tp[c] + 0 + fp[c] + fn[c]) # zeros are True-Negatives, but there are none for this dataset
    f1_score = (2 * precision * recall) / (precision + recall)
    class_metrics[cls_names[c]] = np.array([precision, recall, f1_score, accuracy])
    all_p.append(precision)
    all_r.append(recall)
    all_f1.append(f1_score)
    all_acc.append(accuracy)

Ultralytics YOLOv8.2.28 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8x-cls summary (fused): 133 layers, 56171878 parameters, 0 gradients, 153.8 GFLOPs
train: /kaggle/working/data/train... found 21288 images in 38 classes ✅ 
val: /kaggle/working/data/val... found 5322 images in 38 classes ✅ 
test: /kaggle/working/data/test... found 11405 images in 38 classes ✅ 


test: Scanning /kaggle/working/data/test... 11405 images, 0 corrupt: 100%|██████████| 11405/11405 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 713/713 [00:09<00:00, 74.01it/s]


                   all      0.983          1
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val3


In [28]:
print(all_acc)

[0.9635761589403974, 0.9625, 0.971947194719472, 0.9725085910652921, 0.9506578947368421, 0.9471947194719472, 0.9716088328075709, 0.9459459459459459, 0.9166666666666666, 0.9648562300319489, 0.9615384615384616, 0.9794520547945206, 0.979381443298969, 0.9935275080906149, 0.9539877300613497, 0.9840764331210191, 0.9938271604938271, 0.9526813880126183, 0.9716981132075472, 0.9488817891373802, 0.956953642384106, 0.9515570934256056, 0.9672131147540983, 0.971875, 0.967741935483871, 0.9618055555555556, 0.9754601226993865, 0.9679715302491103, 0.950354609929078, 0.9711538461538461, 0.9675324675324676, 0.9755351681957186, 0.9737704918032787, 1.0, 0.9038461538461539, 0.9968454258675079, 0.975103734439834, 0.9717314487632509]


In [29]:
print(np.mean(all_p))
print(np.mean(all_r))
print(np.mean(all_f1))
print(np.mean(all_acc))

0.9821026753285821
0.9830282113191089
0.98240753503053
0.9656043594006647


In [30]:
# Saving evaluation results

# Define file paths
matrix_file = "/kaggle/working/runs/classify/val3/confusion.csv"
metrics_file = "/kaggle/working/runs/classify/val3/metrics.yaml"
names_file = "/kaggle/working/runs/classify/val3/class_indices.yaml"
tp_fp_fn_file = "/kaggle/working/runs/classify/val3/tp_fp_fn_counts.yaml"

# Save confusion matrix CSV file
import pandas as pd
_ = pd.DataFrame(matrix).to_csv(matrix_file)

# Save YAML files
metrics_export = {k: {'precision': float(v[0]), 'recall': float(v[1]), 'f1-score': float(v[2]), 'accuracy': float(v[3])} for k, v in class_metrics.items()}
metrics_export.update({'Average': {'precision': float(np.mean(all_p)), 'recall': float(np.mean(all_r)), 'f1-score': float(np.mean(all_f1)), 'accuracy': float(np.mean(all_acc))}})
with open(metrics_file, 'w') as met:
    yaml.safe_dump(metrics_export, met)

with open(names_file, 'w') as nf:
    yaml.safe_dump(cls_names, nf)

tp_fp_fn_counts = {int(i): {'tp': int(tp[i]), 'fp': int(fp[i]), 'fn': int(fn[i])} for i in range(cls_count)}
with open(tp_fp_fn_file, 'w') as cnt_f:
    yaml.safe_dump(tp_fp_fn_counts, cnt_f)